In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nwds-batted-balls/sample_submission.csv
/kaggle/input/nwds-batted-balls/train.csv
/kaggle/input/nwds-batted-balls/test.csv


# Disclaimer and accompanying code


first I was going to join some statcast sprint speed and baserunning data and map to player id - I was going to use this to cluster observations based on factors which would predict alignment, eventually I realized it would only work if the file could be accessed from the notebook, not my own file directory. I didn't have time to wuery statcast directy via pybaseball but may soon.


In [2]:
# # Import Libraries
# import pandas as pd
# from pybaseball import playerid_lookup
# import logging
# import re

# # Configure Logging
# logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# def read_csv_file(file_path):
#     """
#     Reads a CSV file into a pandas DataFrame.
#     """
#     try:
#         df = pd.read_csv(file_path)
#         logging.info(f"Successfully read the file: {file_path}")
#         return df
#     except FileNotFoundError:
#         logging.error(f"File not found: {file_path}")
#         raise
#     except Exception as e:
#         logging.error(f"An error occurred while reading the file: {e}")
#         raise

# def clean_names(df):
#     """
#     Cleans and processes the 'entity_name' column:
#     - Splits 'entity_name' into 'Last Name' and 'First Name'
#     - Removes suffixes ('Jr.', 'Sr.', 'II', etc.) from 'Last Name'
#     - Inserts spaces after periods in initials in 'First Name' and 'Last Name'
#     """
#     if 'entity_name' not in df.columns:
#         logging.error("The 'entity_name' column is missing from the DataFrame.")
#         raise KeyError("The 'entity_name' column is required.")
    
#     # Split the 'entity_name' into 'Last Name' and 'First Name'
#     df[['Last Name', 'First Name']] = df['entity_name'].str.split(', ', expand=True)
    
#     # Remove suffixes from 'Last Name'
#     def remove_suffix(name):
#         suffixes = [' Jr.', ' Sr.', ' II', ' III', ' IV', ' V']
#         for suffix in suffixes:
#             if name.endswith(suffix):
#                 name = name[:-len(suffix)]
#                 break  # Assuming only one suffix
#         return name.strip()
    
#     df['Last Name'] = df['Last Name'].apply(remove_suffix)
    
#     # Insert space after periods in initials in 'First Name' and 'Last Name'
#     def insert_space_after_period(name):
#         # Example: 'J.P.' becomes 'J. P.'
#         name = re.sub(r'\b([A-Za-z])\.', r'\1. ', name)
#         # Remove extra spaces if any
#         name = re.sub(r'\s+', ' ', name)
#         return name.strip()
    
#     df['First Name'] = df['First Name'].apply(insert_space_after_period)
#     df['Last Name'] = df['Last Name'].apply(insert_space_after_period)
    
#     logging.info("Cleaned 'entity_name' and extracted 'First Name' and 'Last Name'.")
#     return df

# def get_player_ids(df):
#     """
#     Looks up player IDs using pybaseball and adds them to the DataFrame.
#     Includes checks for availability of player IDs and logs missing entries.
#     """
#     # Get unique names to minimize API calls
#     unique_names = df[['First Name', 'Last Name']].drop_duplicates()
#     unique_names['player_id'] = None  # Initialize the player_id column
    
#     missing_ids = []  # To track names without player IDs
    
#     # Iterate over unique names and look up player IDs
#     for index, row in unique_names.iterrows():
#         first = row['First Name']
#         last = row['Last Name']
#         try:
#             lookup = playerid_lookup(last, first)
#             # Filter players where mlb_played_last is 2024
#             active_players = lookup[lookup['mlb_played_last'] == 2024]
            
#             if len(active_players) == 1:
#                 # Exactly one active player found
#                 player_info = active_players.iloc[0]
#                 player_id = player_info['key_mlbam']
#                 unique_names.at[index, 'player_id'] = player_id
#                 logging.debug(f"Found player ID {player_id} for {first} {last}.")
#             elif len(active_players) > 1:
#                 # Multiple active players found; leave player_id as None
#                 logging.warning(f"Multiple active players found for {first} {last}; player_id left blank.")
#                 missing_ids.append(f"{first} {last}")
#             else:
#                 # No active players found; check if there's only one player in total
#                 if len(lookup) == 1:
#                     player_info = lookup.iloc[0]
#                     player_id = player_info['key_mlbam']
#                     unique_names.at[index, 'player_id'] = player_id
#                     logging.debug(f"Found player ID {player_id} for {first} {last}.")
#                 else:
#                     # No conclusive match; leave player_id as None
#                     logging.warning(f"No conclusive player ID found for {first} {last}.")
#                     missing_ids.append(f"{first} {last}")
#         except Exception as e:
#             logging.error(f"Error looking up {first} {last}: {e}")
#             missing_ids.append(f"{first} {last}")
    
#     # Merge the player IDs back into the original DataFrame
#     df = df.merge(unique_names, on=['First Name', 'Last Name'], how='left')
    
#     # Count and display the number of missing player IDs
#     missing_count = df['player_id'].isna().sum()
#     total_players = len(unique_names)
#     logging.info(f"Total unique players: {total_players}")
#     logging.info(f"Number of players without player ID: {missing_count}")
    
#     if missing_ids:
#         logging.info("Players without IDs:")
#         for name in missing_ids:
#             logging.info(f"- {name}")
    
#     return df

# def main():
#     # File paths
#     base_running_file = r"C:\Users\TrevorWhite\Downloads\base_running.csv"
#     sprint_speed_file = r"C:\Users\TrevorWhite\Downloads\sprint_speed.csv"
#     output_file = r"C:\Users\TrevorWhite\Downloads\combined_data_with_ids.csv"
    
#     # Read the base running CSV file
#     df = read_csv_file(base_running_file)
    
#     # Clean names and extract first and last names
#     df = clean_names(df)
    
#     # Get player IDs and merge them into the DataFrame
#     df = get_player_ids(df)
    
#     # Read the sprint speed CSV file
#     df_sprint = read_csv_file(sprint_speed_file)
#     logging.info("Successfully read the sprint speed data.")
    
#     # Merge the two DataFrames on 'player_id'
#     df_combined = pd.merge(df, df_sprint, on='player_id', how='left')
#     logging.info("Merged base running data with sprint speed data on 'player_id'.")
    
#     # Optional: Handle missing player IDs in the combined DataFrame
#     missing_sprint_data = df_combined[df_combined['player_id'].isna()]
#     if not missing_sprint_data.empty:
#         missing_count = len(missing_sprint_data)
#         logging.info(f"There are {missing_count} records without player_id after merging with sprint speed data.")
    
#     # Save the updated DataFrame to a new CSV file
#     df_combined.to_csv(output_file, index=False)
#     logging.info(f"Combined DataFrame saved to: {output_file}")
    
#     # Display the first few rows of the updated DataFrame
#     print(df_combined.head())

# if __name__ == "__main__":
#     main()


# Loading the data

In [3]:
# Standard libraries
import os
import numpy as np
import pandas as pd
import datetime

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import MaxNLocator

# Machine Learning libraries
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder, LabelBinarizer
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV, cross_val_predict
from sklearn.metrics import roc_auc_score, accuracy_score, silhouette_score
from sklearn.linear_model import RidgeClassifier, RidgeClassifierCV, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.impute import SimpleImputer
from sklearn.base import TransformerMixin, ClassifierMixin, BaseEstimator
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import RFE

# Gradient boosting models
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

# Deep Learning (TensorFlow / Keras)
from tensorflow import keras

# Hyperparameter optimization
import optuna

# Utilities
from tqdm.notebook import tqdm
from itertools import combinations
from scipy.special import softmax
import scipy

# Colorama for colored terminal outputs (optional)
from colorama import Style, Fore

# Configure output display for sklearn pipelines
from sklearn import set_config
set_config(transform_output='pandas')


In [4]:
train = pd.read_csv('/kaggle/input/nwds-batted-balls/train.csv')
test =  pd.read_csv('/kaggle/input/nwds-batted-balls/test.csv')

# Feature Engineering P.1 

not sure who put this out first but I got these from Michael Semenoff and Thiago Mantuani notebooks in this competition

In [5]:
import numpy as np

for df in [train,test]:
    # first new features from initial nb
    # Flip horizontal break for left-handed pitchers (lefties)
    df['pfx_x_adj'] = df['pfx_x'].copy()
    df.loc[df['is_lhp'] == 1, 'pfx_x_adj'] = df.loc[df['is_lhp'] == 1, 'pfx_x_adj'].mul(-1)
    # Adjust spin axis for left-handed pitchers to reflect the same for lefties and righties
    df['spin_axis_adj'] = df['spin_axis'].copy()
    df.loc[df['is_lhp'] == 1, 'spin_axis_adj'] = df.loc[df['is_lhp'] == 1, 'spin_axis_adj'].mul(-1)
    df.loc[df['is_lhp'] == 1, 'spin_axis_adj'] = df.loc[df['is_lhp'] == 1, 'spin_axis_adj'].add(360)
    df.loc[df['is_lhp'] == 1, 'spin_axis_adj'] = df.loc[df['is_lhp'] == 1, 'spin_axis_adj'].replace(360, 0)

    # Adjust plate_x for left-handed batters (flip for consistency with right-handed batters)
    df['plate_x_adj'] = df['plate_x'].copy()
    df.loc[df['is_lhb'] == 1, 'plate_x_adj'] = df.loc[df['is_lhb'] == 1, 'plate_x_adj'].mul(-1)


    df['pitch_speed_diff'] = df['release_speed'] - df['effective_speed']
    df['horizontal_movement'] = df['pfx_x_adj'] * df['release_extension']
    df['vertical_movement'] = df['pfx_z'] * df['release_extension']
    df['pitch_location'] = np.sqrt(df['plate_x']**2 + df['plate_z']**2)
    df['count_pressure'] = (df['balls'] - df['strikes'] + 1)**2
    df['bat_pitch_speed_ratio'] = df['bat_speed'] / df['release_speed']
    df['swing_efficiency'] = df['bat_speed'] / df['swing_length']
    df['movement_complexity'] = np.sqrt(df['pfx_x_adj']**2 + df['pfx_z']**2)
    df['release_angle'] = np.arctan2(df['release_pos_z'], df['release_pos_x'])
    df['approach_angle'] = np.arctan2(df['vz0'], np.sqrt(df['vx0']**2 + df['vy0']**2))
    df['spin_efficiency'] = np.abs(np.sin(np.radians(df['spin_axis_adj'])))
    df['magnus_effect'] = df['release_spin_rate'] * df['movement_complexity']
    df['break_time'] = df['release_extension'] / df['effective_speed']
    df['tunneling_time'] = 0.15 * df['release_speed'] / df['movement_complexity']
    df['stance_advantage'] = ((df['is_lhp'] == 1) & (df['is_lhb'] == 0)) | ((df['is_lhp'] == 0) & (df['is_lhb'] == 1))
    df['pitch_speed_change'] = df.groupby('pitch_number')['release_speed'].diff()
    df['horizontal_change'] = df.groupby('pitch_number')['pfx_x'].diff()
    df['vertical_change'] = df.groupby('pitch_number')['pfx_z'].diff()
    df['baserunner_pressure'] = df['on_1b'] + 2*df['on_2b'] + 3*df['on_3b']
    df['late_inning_pressure'] = (df['inning'] >= 7).astype(int)
    df['release_x_deviation'] = df['release_pos_x'] - df['release_pos_x'].mean()
    df['release_z_deviation'] = df['release_pos_z'] - df['release_pos_z'].mean()
    df['relative_height'] = (df['plate_z'] - df['sz_bot']) / (df['sz_top'] - df['sz_bot'])
    df['is_strike'] = ((df['plate_x'].between(-0.7083, 0.7083)) & 
                       (df['relative_height'].between(0, 1))).astype(int)
    df['contact_quality'] = df['bat_speed'] * np.cos(np.radians(df['spray_angle']))

    # New hitting-focused features in 2nd nb
    df['bat_to_pitch_speed_ratio'] = df['bat_speed'] / df['effective_speed']
    df['swing_plane_efficiency'] = np.cos(np.radians(df['spray_angle'])) / df['swing_length']
    df['contact_zone'] = np.arctan2(df['plate_z'], df['plate_x_adj']) + df['spray_angle']
    df['swing_decision_time'] = df['effective_speed'] / df['swing_length']
    df['power_potential'] = df['bat_speed'] * np.abs(np.sin(np.radians(df['spray_angle'])))
    df['plate_discipline_index'] = df['is_strike'] * df['swing_efficiency']
    df['bat_control'] = df['bat_speed'] / df['swing_length']
    df['exit_velocity_estimate'] = (df['bat_speed'] + df['effective_speed']) * 0.5
    df['launch_angle_estimate'] = df['spray_angle'] * 0.6  # Assuming correlation between spray angle and launch angle
    df['sweet_spot_probability'] = 1 / (1 + np.exp(-(df['bat_speed'] - 70) / 10)) * (1 - np.abs(df['swing_length'] - 24) / 24)

    # New other features used in current leaderboard score

    # Exit velocity to launch angle ratio
    df['exit_velocity_launch_angle_ratio'] = df['exit_velocity_estimate'] / (df['launch_angle_estimate'].abs() + 1)

    # Swing efficiency score
    df['swing_efficiency_score'] = df['swing_efficiency'] * df['bat_control'] * df['sweet_spot_probability']

    # Pitch complexity
    df['pitch_complexity'] = df['release_speed'] * df['spin_efficiency'] * (df['vertical_movement'].abs() + 1)

    # Batter readiness
    df['batter_readiness'] = df['plate_discipline_index'] * df['swing_decision_time']

    # Contact quality score
    df['contact_quality_score'] = df['exit_velocity_estimate'] * df['sweet_spot_probability'] * df['bat_speed']

    # Pitch location effectiveness
    df['pitch_location_effectiveness'] = df['pitch_location'] * df['is_strike'] * (1 - df['plate_discipline_index'])

    # Count pressure impact
    df['count_pressure_impact'] = df['count_pressure'] * (df['balls'] - df['strikes'] + 3)

    # Batter power index
    df['batter_power_index'] = df['power_potential'] * df['exit_velocity_estimate'] * df['bat_speed']

    # Pitch deception score
    df['pitch_deception_score'] = df['effective_speed'] / df['release_speed'] * df['spin_efficiency']

    # Batter contact ability
    df['batter_contact_ability'] = df['bat_control'] * df['contact_zone'] * df['swing_plane_efficiency']

    # Pitch movement complexity
    df['pitch_movement_complexity'] = (df['pfx_z'].abs() + df['vertical_change'].abs()) * df['spin_efficiency']

    # Batter-pitcher matchup advantage
    df['batter_pitcher_matchup'] = df['stance_advantage'] * df['relative_height'] * df['pitch_location_effectiveness']

    # Swing aggressiveness
    df['swing_aggressiveness'] = df['swing_length'] * df['bat_speed'] / df['swing_decision_time']

    # Pitch speed vs bat speed
    df['pitch_bat_speed_differential'] = df['release_speed'] - df['bat_speed']

    # Exit velocity to pitch speed ratio
    df['exit_velocity_pitch_speed_ratio'] = df['exit_velocity_estimate'] / df['release_speed']

    # Vertical approach angle effectiveness
    df['vertical_approach_effectiveness'] = df['approach_angle'] * df['launch_angle_estimate'] * df['exit_velocity_estimate']

    # Horizontal spray angle effectiveness
    df['horizontal_spray_effectiveness'] = df['spray_angle'].abs() * df['exit_velocity_estimate']

    # Strike zone coverage
    df['strike_zone_coverage'] = (df['sz_top'] - df['sz_bot']) * df['plate_discipline_index']

    # Pitch location accuracy
    df['pitch_location_accuracy'] = 1 - (((df['plate_x_adj']**2 + df['plate_z']**2)**0.5) / ((2.5**2 + 2.5**2)**0.5))

# Feature Engineering pt 2. - Courtesy of Nick Wan

The following feature engineering was taken from Nick Wans code 

FIRST SSW AXIS

In [6]:
import pandas as pd
from sklearn.model_selection import KFold
import catboost as cb  # Ensure CatBoost is installed and imported

# Define features and target for spin axis prediction
feats = ['pfx_x_adj', 'pfx_z', 'release_spin_rate']
target = 'spin_axis_adj'

# Step 1: Filter train data to remove rows with missing values in the relevant columns
model_data = train.dropna(subset=feats + [target]).copy()  # Use .copy() to avoid SettingWithCopyWarning

# Step 2: Prepare KFold cross-validation for training
folds = 3
kf = KFold(folds, shuffle=True)
s = pd.Series(dtype=float)  # Initialize an empty series to store predictions

# Step 3: Train CatBoost Regressor model using KFold
for train_idx, test_idx in kf.split(model_data):
    train_fold = model_data.iloc[train_idx]
    test_fold = model_data.iloc[test_idx]
    
    # Train the model
    model = cb.CatBoostRegressor(verbose=False, n_estimators=300)
    model.fit(train_fold.loc[:, feats], train_fold[target])
    
    # Predict on the test fold
    _s = pd.Series(model.predict(test_fold.loc[:, feats]), index=test_fold.index)
    s = pd.concat([s, _s])

# Step 4: Add predicted spin axis values to the dataset
model_data[f'{target}_cb'] = s

# Step 5: Calculate residuals (ssw_axis: actual - predicted)
model_data['ssw_axis'] = model_data[target] - model_data[f'{target}_cb']

# Step 6: Attach predictions and residuals back to the train dataset
train = train.merge(model_data[[f'{target}_cb', 'ssw_axis']], left_index=True, right_index=True, how='left')

# Step 7: Predict for the test dataset
test_before_drop = len(test)  # Record the number of rows before dropping NAs
test = test.dropna(subset=feats).copy()  # Drop rows with missing values for the required features
test_after_drop = len(test)  # Record the number of rows after dropping NAs

# Print the number of records dropped due to missing values
print(f'Number of records dropped from test due to missing values: {test_before_drop - test_after_drop}')

# Use the trained model to predict the spin_axis_adj for the test dataset
test[f'{target}_cb'] = model.predict(test[feats])

# If you want to calculate residuals for test data (optional, as we don't have actual values):
test['ssw_axis'] = test[target] - test[f'{target}_cb']  # Only if actual target is available in the test data

# Now, test contains the predictions


Number of records dropped from test due to missing values: 0


Now SSW via Break


In [7]:
import pandas as pd
from sklearn.model_selection import KFold
import catboost as cb  # Ensure CatBoost is installed and imported

# Step 1: Define features and targets for break prediction
feats = ['spin_axis_adj', 'release_spin_rate']
targets = ['pfx_x_adj', 'pfx_z']

# Step 2: Filter the train dataset to remove rows with missing values in relevant columns
model_data = train.dropna(subset=feats + targets).copy()  # Use .copy() to avoid SettingWithCopyWarning

# Step 3: Prepare KFold cross-validation
folds = 3
kf = KFold(folds, shuffle=True)
s = pd.DataFrame()  # To store predictions

# Step 4: Train CatBoost Regressor model using KFold to predict break
for train_idx, test_idx in kf.split(model_data):
    train_fold = model_data.iloc[train_idx]
    test_fold = model_data.iloc[test_idx]
    
    # Train the model
    model = cb.CatBoostRegressor(verbose=False, n_estimators=300, loss_function='MultiRMSE')
    model.fit(train_fold.loc[:, feats], train_fold[targets])
    
    # Predict on the test fold
    _s = pd.DataFrame(model.predict(test_fold.loc[:, feats]), index=test_fold.index, columns=targets)
    s = pd.concat([s, _s])

# Step 5: Add predicted values and calculate residuals (ssw_x_adj, ssw_z)
for target, col in zip(targets, ['ssw_x_adj', 'ssw_z']):
    model_data[f'{target}_cb'] = s[target]  # Add predicted columns (e.g., pfx_x_adj_cb and pfx_z_cb)
    model_data[col] = model_data[target] - model_data[f'{target}_cb']  # Residuals: actual - predicted

# Step 6: Attach predictions and residuals back to the train dataset
train = train.merge(model_data[[f'pfx_x_adj_cb', f'pfx_z_cb', 'ssw_x_adj', 'ssw_z']], left_index=True, right_index=True, how='left')

# Step 7: Prepare the test set
test_before_drop = len(test)  # Record the number of rows before dropping NAs
test = test.dropna(subset=feats).copy()  # Drop rows with missing values for the required features
test_after_drop = len(test)  # Record the number of rows after dropping NAs

# Print the number of records dropped due to missing values
print(f'Number of records dropped from test due to missing values: {test_before_drop - test_after_drop}')

# Step 8: Use the trained model to predict pfx_x_adj and pfx_z for the test dataset
test_predictions = pd.DataFrame(model.predict(test[feats]), index=test.index, columns=targets)

# Step 9: Store the predictions in the test dataframe with appropriate column names
test['pfx_x_adj_cb'] = test_predictions['pfx_x_adj']
test['pfx_z_cb'] = test_predictions['pfx_z']

# Step 10: Calculate the residuals for the test set (no actual values, so residuals are not useful here, but following the logic)
test['ssw_x_adj'] = test['pfx_x_adj'] - test['pfx_x_adj_cb']
test['ssw_z'] = test['pfx_z'] - test['pfx_z_cb']

# Now, test contains the predictions and residuals


Number of records dropped from test due to missing values: 0


Finally, we also used Nick's code regarding bat speed and swing length vs expected

In [8]:
import pandas as pd
from sklearn.model_selection import KFold
import catboost as cb  # Ensure CatBoost is installed and imported

# Step 1: Mark extra base hits (xbh)
train['xbh'] = 0
train.loc[train['outcome'].isin(['double', 'home_run', 'triple']), 'xbh'] = 1

# Step 2: Filter train data for extra base hits and bat speed > 50 MPH
model_data = train.loc[(train['xbh'] == 1) & (train['bat_speed'] > 50)].copy()  # Use .copy() to avoid warning

# Step 3: Compute averages for batter/pitcher handedness and pitch type
_df = model_data.groupby(['is_lhp', 'is_lhb', 'pitch_type'], as_index=False).agg(
    bat_speed_avg=('bat_speed', 'mean'),
    swing_length_avg=('swing_length', 'mean')
)

# Step 4: Merge the averages back into the original train dataset
train = train.merge(_df, on=['is_lhp', 'is_lhb', 'pitch_type'], how='left')

# Step 5: Ensure there are no missing values in the bat_speed_avg and swing_length_avg columns after merging
train['bat_speed_avg'].fillna(train['bat_speed'].mean(), inplace=True)
train['swing_length_avg'].fillna(train['swing_length'].mean(), inplace=True)

# Step 6: Apply the same logic to the test dataset, merging the same average values
test = test.merge(_df, on=['is_lhp', 'is_lhb', 'pitch_type'], how='left')

# Step 7: Ensure there are no missing values in the test dataset after merging
test['bat_speed_avg'].fillna(test['bat_speed'].mean(), inplace=True)
test['swing_length_avg'].fillna(test['swing_length'].mean(), inplace=True)

# Step 8: **Add the computed averages to the filtered model_data.**
model_data = model_data.merge(_df, on=['is_lhp', 'is_lhb', 'pitch_type'], how='left')

# Step 9: Define features and targets
feats = ['balls', 'strikes', 'plate_x', 'plate_z', 'bat_speed_avg', 'swing_length_avg']
targets = ['bat_speed', 'swing_length']

# Step 10: Prepare KFold cross-validation for train
folds = 3
kf = KFold(folds, shuffle=True)
s1 = pd.DataFrame()

# Step 11: Train CatBoost Regressor model using KFold to predict bat speed and swing length for train
for train_idx, test_idx in kf.split(model_data):
    train_fold = model_data.iloc[train_idx]
    test_fold = model_data.iloc[test_idx]
    
    model = cb.CatBoostRegressor(verbose=False, n_estimators=300, loss_function='MultiRMSE')
    model.fit(train_fold[feats], train_fold[targets])
    
    _s = pd.DataFrame(model.predict(test_fold[feats]), index=test_fold.index, columns=targets)
    s1 = pd.concat([s1, _s])

# Step 12: Predict on the remaining data that wasn't part of model_data
_df = train.loc[~train.index.isin(model_data.index), feats].dropna()
model = cb.CatBoostRegressor(verbose=False, n_estimators=300, loss_function='MultiRMSE')
model.fit(model_data[feats], model_data[targets])
s = pd.DataFrame(model.predict(_df[feats]), index=_df.index, columns=targets)

# Step 13: Attach predictions and calculate residuals for train
for target, col in zip(targets, ['py_bat_speed', 'py_swing_length']):
    model_data[f'{target}_cb'] = s1[target]  # Safe to modify because of .copy()
    train.loc[model_data.index, f'{target}_cb'] = model_data[f'{target}_cb']  # Safely update train using .loc
    train[col] = train[target] - train[f"{target}_cb"]  # Residual calculation

# Step 14: Prepare the test dataset for prediction
test_before_drop = len(test)  # Record the number of rows before dropping NAs
test = test.dropna(subset=feats).copy()  # Drop rows with missing values for the required features
test_after_drop = len(test)

# Print the number of records dropped due to missing values
print(f'Number of records dropped from test due to missing values: {test_before_drop - test_after_drop}')

# Step 15: Use the trained model to predict bat speed and swing length for the test dataset
test_predictions = model.predict(test[feats])

# Step 16: Attach predicted values for test
test['bat_speed_cb'] = test_predictions[:, 0]  # Storing predicted bat_speed
test['swing_length_cb'] = test_predictions[:, 1]  # Storing predicted swing_length

# Step 17: Optionally, calculate residuals for test data (this is optional since we don't have ground truth for test)
test['py_bat_speed'] = test['bat_speed'] - test['bat_speed_cb']
test['py_swing_length'] = test['swing_length'] - test['swing_length_cb']

# Now, both train and test contain the predictions and residuals


/tmp/ipykernel_17/1722895110.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['bat_speed_avg'].fillna(train['bat_speed'].mean(), inplace=True)
/tmp/ipykernel_17/1722895110.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, 

Number of records dropped from test due to missing values: 0


# Feature Engineering Pt 3 - my own ideas

Honestly was just cranking out interactions based on expected relationships

In [9]:
import pandas as pd
import numpy as np

# Assuming train and test are your dataframes
dataframes = [train, test]

# List of pitch types to include
pitch_types = ['4-Seam Fastball', 'Slider', 'Sinker', 'Sweeper', 'Cutter', 'Changeup', 'Curveball', 
               'Split-Finger', 'Slurve', 'Knuckle Curve', 'Eephus', 'Other', 'Forkball', 'Knuckleball', 'Screwball']

for df in dataframes:
    # 1. Transform on_1b, on_2b, on_3b to binary
    df['on_1b'] = df['on_1b'].apply(lambda x: 1 if x != -1 else 0)
    df['on_2b'] = df['on_2b'].apply(lambda x: 1 if x != -1 else 0)
    df['on_3b'] = df['on_3b'].apply(lambda x: 1 if x != -1 else 0)

    # 2. Create pitch type booleans from pitch_name
    for pitch in pitch_types:
        col_name = f'is_{pitch.lower().replace(" ", "_").replace("-", "_")}'
        df[col_name] = df['pitch_name'].apply(lambda x: 1 if x == pitch else 0)

    # 3. Square features: plate_x, sz_top, spray_rate
    df['plate_x_squared'] = df['plate_x_adj'] * abs(df['plate_x_adj'])
    df['sz_top_squared'] = df['sz_top'] ** 2
    df['spray_angle_squared'] = df['spray_angle'] ** 2
    df['swing_length_squared'] = df['swing_length'] ** 2
    df['bat_speed_squared'] = df['bat_speed'] ** 2
    df['pitch_location_squared'] = df['pitch_location'] ** 2
    df['batter_pitcher_matchup_squared'] = df['batter_pitcher_matchup'] ** 2

    # 4. Baserunner pressure * late inning pressure interaction
    df['baserunner_late_inning_pressure'] = df['baserunner_pressure'] * df['late_inning_pressure']

    # 5. Collision Angle Difference Interactions
    df['collision_angle_diff'] = np.abs(df['spray_angle'] - df['approach_angle'])
    df['collision_angle_bat_speed'] = df['collision_angle_diff'] * df['bat_speed']
    df['collision_angle_swing_efficiency'] = df['collision_angle_diff'] * df['swing_efficiency']
    df['collision_angle_swing_plane_efficiency'] = df['collision_angle_diff'] * df['swing_plane_efficiency']

    # 6. Power Potential * Swing Length
    df['power_swing_length'] = df['power_potential'] * df['swing_length']

    # 7. Spray Angle * Pitch Type Booleans (only for specified pitch types)
    for col in df.columns:
        if col.startswith('is_') and any(pitch.lower().replace(" ", "_").replace("-", "_") in col for pitch in pitch_types):
            df[f'spray_angle_{col}'] = df['spray_angle'] * df[col]

    # 8. Swing Length * Pitch Type Booleans * Count Pressure * Plate X * is_lhb
    for col in df.columns:
        if col.startswith('is_') and any(pitch.lower().replace(" ", "_").replace("-", "_") in col for pitch in pitch_types):
            df[f'swing_length_{col}_interaction'] = (df['swing_length'] * df[col] * 
                                                     df['count_pressure'] * df['plate_x_adj'] * 
                                                     df['batter_pitcher_matchup_squared'])

    # 9. Power Potential, Contact Zone, and Release Spin Rate * Pitch Type Booleans (Separately)
    for col in df.columns:
        if col.startswith('is_') and any(pitch.lower().replace(" ", "_").replace("-", "_") in col for pitch in pitch_types):
            df[f'power_potential_{col}_interaction'] = df['power_potential'] * df[col]
            df[f'contact_zone_{col}_interaction'] = df['contact_zone'] * df[col]
            df[f'release_spin_rate_{col}_interaction'] = df['release_spin_rate'] * df[col]

    # 10. Power Potential * Bat Speed * Heart Zone
    df['power_potential_bat_speed_heart'] = df['power_potential'] * df['bat_speed'] * df['pitch_location']

    # 11. List of pitch attributes to create interactions with pitch types (for the specified pitch types)
    pitch_attributes = ['plate_x_adj', 'plate_z', 'pitch_speed_diff', 'horizontal_movement', 
                        'vertical_movement', 'pitch_location', 'release_angle', 
                        'approach_angle', 'spin_efficiency', 'magnus_effect', 
                        'break_time', 'tunneling_time', 'spin_axis_adj', 'release_speed', 'effective_speed', 
                        'release_spin_rate', 'ssw_x_adj', 'ssw_z', 'spin_axis_adj_cb']

    for col in df.columns:
        if col.startswith('is_') and any(pitch.lower().replace(" ", "_").replace("-", "_") in col for pitch in pitch_types):
            for attr in pitch_attributes:
                df[f'{col}_{attr}_interaction'] = df[col] * df[attr]

    # Create the top_rail feature for pitches with plate_z >= 41 inches (3.4167 feet)
    df['top_rail'] = (df['plate_z'] >= 3.4167).astype(int)

    # Create a binary feature 'is_9th' where it is 1 if the inning is 9, and 0 otherwise
    df['is_9th'] = (df['inning'] == 9).astype(int)

    # Interaction between py_bat_speed_cb and other variables
    df['py_bat_speed__interaction'] = df['py_bat_speed'] * df['count_pressure'] * df['plate_x_adj'] * df['plate_z']

    # Interaction between py_swing_length_cb and other variables
    df['py_swing_length__interaction'] = df['py_swing_length'] * df['count_pressure'] * df['plate_x_adj'] * df['plate_z']

#     # Fill missing values for each column with 0
#     df['runner_on_First_aoa'].fillna(0, inplace=True)
#     df['runner_on_First_safe_Rate'].fillna(0, inplace=True)
#     df['runner_on_First_ss'].fillna(0, inplace=True)
    
#     df['runner_on_second_aoa'].fillna(0, inplace=True)
#     df['runner_on_second_safe_Rate'].fillna(0, inplace=True)
#     df['runner_on_second_ss'].fillna(0, inplace=True)
    
#     df['runner_on_third_aoa'].fillna(0, inplace=True)
#     df['runner_on_third_safe_Rate'].fillna(0, inplace=True)
#     df['runner_on_third_ss'].fillna(0, inplace=True)



/tmp/ipykernel_17/4258116916.py:60: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'release_spin_rate_{col}_interaction'] = df['release_spin_rate'] * df[col]
/tmp/ipykernel_17/4258116916.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'power_potential_{col}_interaction'] = df['power_potential'] * df[col]
/tmp/ipykernel_17/4258116916.py:59: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at o

# Code used to cluster (did not use, see disclaimer)



In [10]:
# import pandas as pd
# from sklearn.preprocessing import StandardScaler
# from sklearn.cluster import KMeans
# from sklearn.metrics import silhouette_score
# import matplotlib.pyplot as plt

# # Step 1: Prepare relevant features for clustering
# features = [
#     'runner_on_First_aoa', 'runner_on_First_safe_Rate', 'runner_on_First_ss', 
#     'runner_on_second_aoa', 'runner_on_second_safe_Rate', 'runner_on_second_ss', 
#     'runner_on_third_aoa', 'runner_on_third_safe_Rate', 'runner_on_third_ss', 
#     'batter_pitcher_matchup', 'count_pressure', 'on_1b', 'on_2b', 'on_3b', 
#     'late_inning_pressure', 'sz_top_squared', 'is_lhb', 'is_9th'
# ]

# # Step 2: Drop rows with missing values in the selected features from train and test
# train_before_drop = len(train)
# train_clustering = train.dropna(subset=features)
# train_after_drop = len(train_clustering)
# dropped_train_rows = train_before_drop - train_after_drop


# test_before_drop = len(test)
# test_clustering = test.dropna(subset=features)
# test_after_drop = len(test_clustering)
# dropped_test_rows = test_before_drop - test_after_drop

# print(f'Number of rows dropped from test:{test_before_drop} {test_after_drop}')
# print(f'Number of rows dropped from test: {dropped_test_rows}')

# # Step 3: Standardize the features for clustering
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(train_clustering[features])
# X_test_scaled = scaler.transform(test_clustering[features])

# # Step 4: Iterative process to determine the optimal number of clusters using KMeans
# def find_optimal_clusters(X, max_clusters=8):
#     inertia = []
#     silhouette_scores = []
#     K_range = range(2, max_clusters + 1)

#     for n_clusters in K_range:
#         # Apply KMeans clustering
#         kmeans = KMeans(n_clusters=n_clusters, random_state=42)
#         kmeans.fit(X)
        
#         # Append inertia (sum of squared distances to the nearest cluster center)
#         inertia.append(kmeans.inertia_)
        
#         # Compute silhouette score to evaluate separation between clusters
#         silhouette_avg = silhouette_score(X, kmeans.labels_)
#         silhouette_scores.append(silhouette_avg)

#     # Plot the Elbow Method graph
#     plt.figure(figsize=(10, 5))
#     plt.plot(K_range, inertia, 'bo-', label='Inertia (Elbow Method)')
#     plt.xlabel('Number of clusters (K)')
#     plt.ylabel('Inertia')
#     plt.title('Elbow Method for Optimal Clusters')
#     plt.legend()
#     plt.grid(True)
#     plt.show()

#     # Plot the silhouette score graph
#     plt.figure(figsize=(10, 5))
#     plt.plot(K_range, silhouette_scores, 'go-', label='Silhouette Score')
#     plt.xlabel('Number of clusters (K)')
#     plt.ylabel('Silhouette Score')
#     plt.title('Silhouette Score for Optimal Clusters')
#     plt.legend()
#     plt.grid(True)
#     plt.show()

#     # Return the range of K, inertia, and silhouette scores for further analysis
#     return K_range, inertia, silhouette_scores


In [11]:
# import pandas as pd
# from sklearn.preprocessing import StandardScaler
# from sklearn.cluster import KMeans
# import matplotlib.pyplot as plt

# # Step 1: Prepare relevant features for clustering
# features = [
#     'runner_on_First_aoa', 'runner_on_First_safe_Rate', 'runner_on_First_ss', 
#     'runner_on_second_aoa', 'runner_on_second_safe_Rate', 'runner_on_second_ss', 
#     'runner_on_third_aoa', 'runner_on_third_safe_Rate', 'runner_on_third_ss', 
#     'batter_pitcher_matchup', 'count_pressure', 'on_1b', 'on_2b', 'on_3b', 
#     'late_inning_pressure', 'sz_top_squared', 'is_lhb', 'is_9th'
# ]

# # Step 2: Drop rows with missing values in the selected features from train and test
# train_before_drop = len(train)
# train_clustering = train.dropna(subset=features)
# train_after_drop = len(train_clustering)
# dropped_train_rows = train_before_drop - train_after_drop

# test_before_drop = len(test)
# test_clustering = test.dropna(subset=features)
# test_after_drop = len(test_clustering)
# dropped_test_rows = test_before_drop - test_after_drop

# print(f'Number of rows dropped from train: {dropped_train_rows}')
# print(f'Number of rows dropped from test: {dropped_test_rows}')

# # Step 3: Standardize the features for clustering
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(train_clustering[features])
# X_test_scaled = scaler.transform(test_clustering[features])

# # Step 4: Apply KMeans clustering with the fixed number of clusters (7 clusters)
# forced_clusters = 7
# kmeans_forced = KMeans(n_clusters=forced_clusters, random_state=42)

# # Step 5: Apply KMeans with the forced number of clusters on the train dataset
# train_clustering['fielder_positioning_cluster'] = kmeans_forced.fit_predict(X_train_scaled)

# # Step 6: Attach cluster labels to the original train dataset
# train['fielder_positioning_cluster'] = train_clustering['fielder_positioning_cluster']

# # Step 7: Use the trained KMeans model to predict clusters for the test dataset
# test_clustering['fielder_positioning_cluster'] = kmeans_forced.predict(X_test_scaled)

# # Step 8: Attach cluster labels to the original test dataset
# test['fielder_positioning_cluster'] = test_clustering['fielder_positioning_cluster']

# # Step 9: Review clustering results for the train and test sets
# print("Train cluster counts:\n", train['fielder_positioning_cluster'].value_counts())
# print("Test cluster counts:\n", test['fielder_positioning_cluster'].value_counts())



# Model Creation + Tuning


Sadly, I'm actively running out of time to tune this thing, would prefer to standardize scales and tune it fully but hopefully I will get a successful model submitted before the deadline, but it might not converge in time

In [12]:
import pandas as pd
import numpy as np
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Load dataset
df = train  # Assuming 'train' is the full dataset that includes both training and testing data

# Ensure 'uid' is in the dataframe before starting
assert 'uid' in df.columns, "'uid' column is missing from the original dataframe"

# Define the columns to exclude to prevent target leakage (if applicable)
exclude_columns = ['pitch_type', 'game_type', 'game_year', 'pitch_number', 'pitch_name', 'outcome', 'xbh']

# Select all columns except the ones in exclude_columns, but ensure 'uid' and 'outcome_code' are not dropped
df_model = df.loc[:, ~df.columns.isin(exclude_columns)]

# Ensure 'uid' and 'outcome_code' are still present
assert 'uid' in df_model.columns, "'uid' column was dropped during feature selection"
assert 'outcome_code' in df_model.columns, "'outcome_code' column was dropped during feature selection"

# Define features (excluding 'uid' and the target column)
feats = df_model.drop(columns=['uid', 'outcome_code']).columns.tolist()

# Target for the model
target = 'outcome_code'

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test, uid_train, uid_test = train_test_split(
    df_model[feats], df[target], df_model['uid'], test_size=0.2, random_state=42
)

# Initialize CatBoostClassifier with a low number of iterations to speed up training
model = cb.CatBoostClassifier(
    iterations=100,  # Lower iterations for faster training
    depth=6,  # Simpler depth for faster training
    learning_rate=0.1,  # Standard learning rate
    verbose=0,  # Suppress detailed output
    loss_function='MultiClass', 
    eval_metric='AUC'
)

# Fit the model on the training data
model.fit(X_train, y_train)

# Predict probabilities for the test set
pred_proba_test = model.predict_proba(X_test)

# Ensure the probabilities sum to 1 for each instance
assert np.allclose(pred_proba_test.sum(axis=1), 1), "Predicted probabilities do not sum to 1!"

# Convert the predicted probabilities into a DataFrame
outputs = pd.DataFrame(pred_proba_test, columns=['out', 'single', 'double', 'triple', 'home_run'], index=uid_test)

# Evaluate model performance using ROC AUC on the test set
roc_auc = roc_auc_score(y_test, pred_proba_test, multi_class='ovo')
print(f'ROC AUC Score on Test Set: {roc_auc}')


ROC AUC Score on Test Set: 0.7094299698651078


# Submission
first time using kaggle for this type of competition, through me for a loop lol

In [13]:
df_test = test  # Replace 'test' with your actual test DataFrame

# Ensure 'uid' is still present in the test set
assert 'uid' in df_test.columns, "'uid' column is missing from the test dataframe"

# Include 'uid' with features for tracking, but it will be removed before prediction
X_test_actual = df_test[feats + ['uid']]  # 'feats' is the list of feature columns used for training

# Extract 'uid' to keep it for the final submission
uid_test_actual = X_test_actual['uid']

# Drop 'uid' from the features before prediction, since it should not be used in the model
X_test_actual = X_test_actual.drop(columns=['uid'])

# Predict on the actual test set using the trained model
pred_proba_test_actual = model.predict_proba(X_test_actual)

# Convert the predicted probabilities into a DataFrame for the actual test set
outputs_actual = pd.DataFrame(pred_proba_test_actual, columns=['out', 'single', 'double', 'triple', 'home_run'])

# Rejoin 'uid' with the predicted probabilities
outputs_actual = pd.concat([uid_test_actual.reset_index(drop=True), outputs_actual], axis=1)

# Create the final submission DataFrame with 'uid' and predicted probabilities
submission = pd.DataFrame({
    'uid': outputs_actual['uid'],
    'out': outputs_actual['out'],
    'single': outputs_actual['single'],
    'double': outputs_actual['double'],
    'triple': outputs_actual['triple'],
    'home_run': outputs_actual['home_run']
})

# Save the submission file
submission.to_csv('submission.csv', index=False)
print("Submission file created: 'submission.csv'")

# Optional: Save the model for future use
model.save_model('fast_catboost_model_with_test_split.cbm')


Submission file created: 'submission.csv'
